## XDAWN Decoding from EEG data
* XDAWN converts channels and events to create feature vectors that can be fed into a logistic regression ([MNE documentation](https://www.martinos.org/mne/stable/auto_examples/decoding/plot_decoding_xdawn_eeg.html))
* This is a first attempt at exploring machine learning with EEG data. This method will produce a confusion matrix of event_ids.
* Note: This code is NOT mine, but is taken from the link above and adjusted for data used in project.  

In [29]:
# Authors: Alexandre Barachant <alexandre.barachant@gmail.com>
#
# License: BSD (3-clause)

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

import mne
from mne import io, pick_types, read_events, Epochs
# from mne.datasets import sample
from mne.preprocessing import Xdawn
from mne.decoding import Vectorizer
from mne.viz import tight_layout

import os
import os.path as op

#data path for each run of each subject.
drive_data_path = 'E:\eeg_data'

#data path on my external hdd for folder containing all tests of each subject in one file
drive_all_data_path = 'E:\eeg_data\ica_140_500_0.1'



The documentations starts with raw data, however we alreaday have epoched data with events and event id's. 

In [38]:
epochs = mne.read_epochs(op.join(drive_data_path, 'Ex10_Suj1_Run1-epo.fif'),
                          preload=True)

Reading E:\eeg_data\Ex10_Suj1_Run1-epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 61) active
    Found the data of interest:
        t =       0.00 ...    1400.00 ms
        0 CTF compensation matrices available
280 matching events found
Created an SSP operator (subspace dimension = 1)
280 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [47]:
#create classification pipeline
clf = make_pipeline(Xdawn(n_components = 3),
                    Vectorizer(),
                    MinMaxScaler(),
                    LogisticRegression(penalty='l1'))

In [50]:
#cross validator
cv = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)


#Do cross-validation
labels = epochs.events[:,-1]
preds = np.empty(len(labels))
for train, test in cv.split(epochs, labels):
    clf.fit(epochs[train], labels[train])
    preds[test] = clf.predict(epochs[test])

ValueError: Could not compute eigenvalues, ensure proper regularization (the leading minor of order 61 of 'b' is not positive definite. The factorization of 'b' could not be completed and no eigenvalues or eigenvectors were computed.)

In [27]:
type(epochs)

mne.epochs.Epochs

In [28]:
type(epoch)

mne.epochs.Epochs